In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

In [7]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]


class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights


class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
    
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels


def update_weights(gradients, weights):
    learning_rate = 0.001
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)


def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions
        )
        per_sample_accuracy = tf.keras.metrics.sparse_categorical_accuracy(
            labels_batch, predictions
        )
        average_loss = tf.reduce_mean(per_sample_losses)
        average_accuracy = tf.reduce_mean(per_sample_accuracy)

    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss, average_accuracy

def fit(model, images, labels, epochs, batch_size=128):
    for epoch in range(epochs):
        print('Epoch %d' % epoch)
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch in range(len(images) // batch_size):
            images_batch, labels_batch = batch_generator.next()
            loss, accuracy = one_training_step(model, images_batch, labels_batch)
            if batch % 100 == 0:
                print('loss at batch %d: %.2f' % (batch, loss))
                print('accuracy at batch %d: %.2f' % (batch, accuracy))

In [8]:
model = NaiveSequential([
    NaiveDense(input_size=784, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert(len(model.weights) == 4)

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 784))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 784))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print('accuracy: %.2f' % np.average(matches))

Epoch 0
loss at batch 0: 3.80
accuracy at batch 0: 0.10
loss at batch 100: 2.24
accuracy at batch 100: 0.27
loss at batch 200: 2.24
accuracy at batch 200: 0.11
loss at batch 300: 2.10
accuracy at batch 300: 0.22
loss at batch 400: 2.23
accuracy at batch 400: 0.31
Epoch 1
loss at batch 0: 1.94
accuracy at batch 0: 0.58
loss at batch 100: 1.88
accuracy at batch 100: 0.62
loss at batch 200: 1.86
accuracy at batch 200: 0.27
loss at batch 300: 1.72
accuracy at batch 300: 0.63
loss at batch 400: 1.84
accuracy at batch 400: 0.47
Epoch 2
loss at batch 0: 1.61
accuracy at batch 0: 0.76
loss at batch 100: 1.58
accuracy at batch 100: 0.77
loss at batch 200: 1.52
accuracy at batch 200: 0.57
loss at batch 300: 1.43
accuracy at batch 300: 0.78
loss at batch 400: 1.53
accuracy at batch 400: 0.61
Epoch 3
loss at batch 0: 1.33
accuracy at batch 0: 0.81
loss at batch 100: 1.34
accuracy at batch 100: 0.77
loss at batch 200: 1.25
accuracy at batch 200: 0.77
loss at batch 300: 1.21
accuracy at batch 300: 0